In [1]:
## library imports

import pandas as pd
import config
from pivoting import create_game

In [2]:
## db engine

engine = config.get_engine(database='processingdev')

In [3]:
sql = """

        SELECT
             LinescoreID
            ,GameID
            ,EventID
            ,EventName
            ,Draw
            ,End1
            ,End2
            ,End3
            ,End4
            ,End5
            ,End6
            ,End7
            ,End8
            ,End9
            ,End10
            ,End11
            ,End12
            ,Hammer
            ,FinalScore
            ,Team AS TeamName
            ,URL
            ,DrawNum
            ,Year
            ,StartMonth
            ,EndMonth
            ,EventDayStart
            ,EventDayENd
            ,CONCAT(LinescoreID,'|',Team) AS Team
        FROM ValidatedScores

"""

In [4]:
## read in scores
scores = pd.read_sql(sql,con=engine)

In [5]:
scores

,LinescoreID,GameID,EventID,EventName,Draw,End1,End2,End3,End4,End5,...,FinalScore,TeamName,URL,DrawNum,Year,StartMonth,EndMonth,EventDayStart,EventDayENd,Team
0,1,1,3000,Swiss Curling League,1,0.0,0.0,1.0,0.0,1.0,...,3,Marc Suter,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3,1|Marc Suter
1,2,1,3000,Swiss Curling League,1,0.0,2.0,0.0,3.0,0.0,...,9,Peter de Cruz,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3,2|Peter de Cruz
2,3,2,3000,Swiss Curling League,1,2.0,0.0,0.0,0.0,0.0,...,7,Sven Michel,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3,3|Sven Michel
3,4,2,3000,Swiss Curling League,1,0.0,0.0,2.0,1.0,0.0,...,5,Alexander Attinger,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3,4|Alexander Attinger
4,5,3,3000,Swiss Curling League,1,1.0,0.0,3.0,0.0,1.0,...,11,St. Gallen Bar,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3,5|St. Gallen Bar
5,6,3,3000,Swiss Curling League,1,0.0,3.0,0.0,2.0,0.0,...,5,Roman Ruch,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3,6|Roman Ruch
6,7,4,3000,Swiss Curling League,2,2.0,2.0,0.0,0.0,1.0,...,7,Bernhard Werthemann,https://www.curlingzone.com/event.php?eventid=...,2,2013,Jan,Feb,11,3,7|Bernhard Werthemann
7,8,4,3000,Swiss Curling League,2,0.0,0.0,1.0,0.0,0.0,...,5,Stefan Haesler,https://www.curlingzone.com/event.php?eventid=...,2,2013,Jan,Feb,11,3,8|Stefan Haesler
8,9,5,3000,Swiss Curling League,2,0.0,0.0,1.0,1.0,0.0,...,6,Michael Schifferli,https://www.curlingzone.com/event.php?eventid=...,2,2013,Jan,Feb,11,3,9|Michael Schifferli
9,10,5,3000,Swiss Curling League,2,0.0,0.0,0.0,0.0,2.0,...,4,Marc Pfister,https://www.curlingzone.com/event.php?eventid=...,2,2013,Jan,Feb,11,3,10|Marc Pfister


In [6]:
## push pivoted scores to new table 

index = False
if_exists='append'

for game_id in scores.GameID.unique():
    try:
        game = create_game(game_id=game_id,games=scores)
        game.to_frame().to_sql("PivotedScores",con=engine,index=index,if_exists=if_exists)
    except Exception as e:
        df = scores.loc[scores.GameID == game_id].copy()
        df['ExceptionType'] = e.__repr__()
        df.to_sql("PivotingExceptions",con=engine,index=index,if_exists=if_exists)